[## 文档链接](https://huggingface.co/docs/transformers/main/en/main_classes/quantization#quantization)

## GPTQ

### GPTQ：面向预训练 Transformer 模型设计的量化技术（ICLR 2023）
GPTQ：Accurate Post-Training Quantization for Generative Pre-trained Transformers 是一个高效、精准的
量化技术，特别适用于大规模GPT模型，能够在显著降低模型大小和计算需求的同时，保持高准确度和推理速度。
***
GPTQ算法具有以下技术特点：\
\
1.专为GPT模型设计：GPTQ针对大规模GPT模型（如1750亿参数规模的模型）进行优化，解决了这类模型因
规模庞大导致的高计算和存储成本问题。\
2.一次性权重量化方法：GPTQ是一种基于近似二阶信息的权重量化方法，能够在一次处理中完成模型的量化。\
3.高效率：GPTQ能在大约四个GPU小时内完成1750亿参数的GPT模型的量化。\
4.低位宽量化：通过将权重位宽降至每个权重3或4位，GPTQ显著减少了模型的大小。\
5.准确度保持：即便在进行显著的位宽减少后，GPTQ也能保持与未压缩模型相近的准确度，减少性能损失。\
6.支持极端量化：GPTQ还可以实现更极端的量化，如2位或三元量化，同时保持合理的准确度。\
7.推理速度提升：使用GPTQ量化的模型在高端GPU（如NVIDIA A100）上实现了大约3.25倍的推理速度提升，
在成本效益更高的GPU（如NVIDIA A6000）上实现了大约4.5倍的速度提升。\
8.适用于单GPU环境：GPTQ使得在单个GPU内执行大规模模型的生成推理成为可能，显著降低了部署这类模
型的硬件要求。

![image](../static/微信图片_20240304084405.png)

### GPTQ 量化算法核心流程
核心步骤：使用存储在Cholesky（切尔斯基）分解中的逆Hessian（海森）
信息量化连续列的块（加粗表示），并在步骤结束时更新剩余的权重
（蓝色表示），在每个块内递归（白色中间块）地应用量化过程。\
GPTQ量化过程的关键步骤操作，具体描述如下：\
1.块量化：选择一块连续的列（在图中加粗表示），并将其作为当前步骤
的量化目标。\
2.使用Cholesky分解：利用Cholesky分解得到的逆Hessian信息来量化选定的块。Cholesky分解提供了一种数值稳定的方法来处理逆矩阵，这对于维
持量化过程的准确性至关重要。\
3.权重更新：在每个量化步骤的最后，更新剩余的权重（在图中以蓝色表
示）。这个步骤确保了整个量化过程的连贯性和精确性。\
4.递归量化：在每个选定的块内部，量化过程是递归应用的。这意味着量
化过程首先聚焦于一个较小的子块，然后逐步扩展到整个块。
通过这种方式，GPTQ方法能够在保持高度精度的同时，高效地处理大量
的权重，这对于大型模型的量化至关重要。这种策略特别适用于处理大
型、复杂的模型，如GPT系列，其中权重数量巨大，且量化过程需要特别
小心以避免精度损失。

## AWQ

### 激活感知权重量化（Activation-aware Weight Quantization, AWQ）
激活感知权重量化（AWQ）算法，其原理不是对模型中的所有权重进行量化，而是仅保留小部分（1%）对LLM性能\
至关重要的权重。其算法主要特点如下：\
1.低位权重量化：AWQ专为大型语言模型（LLMs）设计，支持低位（即少位数）的权重量化，有效减少模型大小。\
2.重点保护显著权重：AWQ基于权重重要性不均的观察，只需保护大约1%的显著权重，即可显著减少量化误差。\
3.观察激活而非权重：在确定哪些权重是显著的过程中，AWQ通过观察激活分布而非权重分布来进行。\
4.无需反向传播或重构：AWQ不依赖于复杂的反向传播或重构过程，因此能够更好地保持模型的泛化能力，避免对
特定数据集的过拟合。\
5.适用于多种模型和任务：AWQ在多种语言建模任务和领域特定基准测试中表现出色，包括指令调整的语言模型和
多模态语言模型。\
6.高效的推理框架：与AWQ配套的是一个为LLMs量身定做的高效推理框架，提供显著的速度提升，适用于桌面和
移动GPU。\
7.支持边缘设备部署：这种方法支持在内存和计算能力有限的边缘设备（如NVIDIA Jetson Orin 64GB）上部署大
型模型，如70B Llama-2模型。

![image](../static/微信图片_20240304092755.png)

## 比较

![image](../static/微信图片_20240304093236.png)

## BAB


### BitsAndBytes 简介
BitsAndBytes（BNB）是自定义CUDA函数的轻量级包装器，特别是8比特优化器、矩阵乘法和量
化函数。主要特征如下：\
 \
•具有混合精度分解的8比特矩阵乘法\
•LLM.int8()推理\
•8比特优化器：Adam、AdamW、RMSProp、LARS、LAMB、Lion（节省75%内存）\
•稳定的嵌入层：通过更好的初始化和标准化提高稳定性\
•8比特量化：分位数、线性和动态量化\
•快速的分位数估计：比其他算法快100倍\
 \
在 Transformers 量化方案中，BNB 是将模型量化为8位和4位的最简单选择。\
 \
•8位量化将fp16中的异常值与int8中的非异常值相乘，将非异常值转换回fp16，然后将它们相加以
返回fp16中的权重。这减少了异常值对模型性能产生的降级效果。\
•4位量化进一步压缩了模型，并且通常与QLoRA一起用于微调量化LLM（低精度语言模型）。

## Half

In [1]:
from transformers import AutoTokenizer,AutoModel,AutoModelForCausalLM
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=AutoModelForCausalLM.from_pretrained('E:\model\language\opt-125m',trust_remote_code=True,device_map='auto').to(device)

# 总结 AWQ yyds

# 代码环节

***

### AWQ

![image](../static/微信图片_20240305102346.png)

In [18]:
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoConfig,AwqConfig
import torch
model_path='/mnt/data/opt-125m'
quant_path='/mnt/phb/quant'
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
device

device(type='cuda')

In [20]:
model=AutoModelForCausalLM.from_pretrained(model_path,trust_remote_code=True).to(device)
tokenizer=AutoTokenizer.from_pretrained(model_path)

In [21]:
quant_config=AwqConfig(
    bits=4,
    group_size=128,
    zero_point=True,
    version='gemm'
).to_dict()
model.model.config.quantization_config=quant_config

In [22]:
model.save_pretrained(quant_path)
tokenizer.save_pretrained(quant_path)

('/mnt/phb/quant/tokenizer_config.json',
 '/mnt/phb/quant/special_tokens_map.json',
 '/mnt/phb/quant/vocab.json',
 '/mnt/phb/quant/merges.txt',
 '/mnt/phb/quant/added_tokens.json',
 '/mnt/phb/quant/tokenizer.json')

In [23]:
model.eval

<bound method Module.eval of OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_laye

## GPTQ

![image](../static/微信图片_20240305102351.png)

In [14]:
from transformers import AutoModelForCausalLM,AutoTokenizer,GPTQConfig
import torch
model_path='/mnt/data/opt-125m'
quant_path='/mnt/phb/quant'
quant_config=GPTQConfig(
    bits=4,
    group_size=128,
    dataset=["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."],
    desc_act=False,
)



In [15]:
tokenizer=AutoTokenizer.from_pretrained(model_path,trust_remote_code=True)
model=AutoModelForCausalLM.from_pretrained(model_path,quantization_config=quant_config,trust_remote_code=True).to(device)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Quantizing model.decoder.layers blocks : 100%|██████████| 12/12 [00:42<00:00,  3.57s/it]
Found modules on cpu/disk. Using Exllama/Exllamav2 backend requires all the modules to be on GPU. Setting `disable_exllama=True`
/opt/conda/envs/phb/lib/python3.10/site-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [17]:
model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[ 1712808666, -1248295259, -2025411892,  ..., -1486452502,
                         2019142072, -1735820810],
                       [-2000132747,  -578262345,  1484081337,  ..., -1230600537,
                        -2019252056, -2023311003],
                       [ -710293850, -1153090188,  1431922298,  ..., -1768449094,
                        -1984337253,  2022406582],
                       ...,
                       [-1451935592, -1494580055, -1772844344,  ..., -1517635426,
                         -664417400,  -409622870],
                       [-2007473565,  1218733898,  1737251004,  ...,  1741199510,
                        -1732560249, -1754850968],
                       [ 1999202918, -1986294939,  1737140825,  ..., -1461086871,
                        -1450465416, -1756087955]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

## BNB

![image](../static/微信图片_20240305102356.png)

# 调用量化模型

In [24]:
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoConfig,AwqConfig
import torch
model_path='/mnt/data/opt-125m'
quant_path='/mnt/phb/quant'
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [25]:

print(torch.__version__)
device

2.2.1+cu121


device(type='cuda')

In [26]:
model=AutoModelForCausalLM.from_pretrained(quant_path,trust_remote_code=True).to(device)
tokenizer=AutoTokenizer.from_pretrained(quant_path,trust_remote_code=True)



You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of the model checkpoint at /mnt/phb/quant were not used when initializing OPTForCausalLM: ['model.decoder.layers.0.fc1.weight', 'model.decoder.layers.0.fc2.weight', 'model.decoder.layers.0.self_attn.k_proj.weight', 'model.decoder.layers.0.self_attn.out_proj.weight', 'model.decoder.layers.0.self_attn.q_proj.weight', 'model.decoder.layers.0.self_attn.v_proj.weight', 'model.decoder.layers.1.fc1.weight', 'model.decoder.layers.1.fc2.weight', 'model.decoder.layers.1.self_attn.k_proj.weight', 'model.decoder.layers.1.self_attn.out_proj.weight', 'model.decoder.layers.1.self_attn.q_proj.weight', 'model.decoder.layers.1.self_attn.v_proj.weight', 'model.decoder.layers.10.fc1.weight', 'model.decoder.layers.10.fc2.weight', 'model.decoder.layers.10.self_attn.k_proj.weight', 'mo

In [27]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    print(inputs)
    out = model.generate(**inputs, max_new_tokens=64)
    print(out)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [29]:
generate_text('hello')

{'input_ids': tensor([[    2, 42891]], device='cuda:0'), 'attention_mask': tensor([[1, 1]], device='cuda:0')}


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
